## Genome informatics
# RNA-Seq analysis : Quantification

## RNA-Seq quantification

Raw counting or probabilistic estimation dilemma is (mostly) about doing analysis on gene vs transcript level.
- When quantifying on **gene** level - we simply count the number of reads aligned to each gene.
- On **transcript** level, we need an algorithm such as the Expectation Maximization (EM, pictured below) to deal with the uncertainty.

<br>
<img src="em1.png" width="550">

<img src="em2.png" width="550">

When performing statistical analysis of RNA expression, doing it on gene level compared to transcript level is more robust and experimentally actionable. The biologists will also usually draw their conclusions on gene level since that's the level that the biological pathways are annotated on. 

However, the use of gene counts for statistical analysis can mask transcript-level dynamics. A popular alternative nowadays is to estimate the transcript abundances and then aggregate to gene level, or perform the entire analysis on trancsript level (testing for differential expression) and then aggregate the results.

## Gene level quantification

We shall, for simplicity's sake, only perform gene level quantification. Even though it has some drawbacks, it's a strategy still used by most genomic scientists. Here's what we have at the beginning of the quantification step and what we want to estimate:

- We **have**: aligned reads. A file format called SAM/BAM is now the standard formats for next-generation sequence alignments.
- We **estimate**: relative abundance.

We'll start with some simple methods to compare two features. We say that 2 intervals overlap if any region of the two features cover the same position.

In [1]:
def overlap(x, y):
    """
    This function takes two intervals and determines whether 
    they have any overlapping segments.
    """
    new_start = max(x[1], y[1])
    new_end = min(x[2], y[2])
    
    if new_start < new_end and x[0] == y[0]:
        return True
    return False

We will define four intervals as tuples.

In [2]:
region1 = ("chr3", 27, 82)
region2 = ("chr4", 27, 82)
region3 = ("chr3", 57, 75)
region4 = ("chr3", 85, 107)

In [3]:
overlap(region1, region2)

False

In [4]:
overlap(region1, region3)

True

In [5]:
overlap(region1, region4)

False

We shall now pick a gene of interest and try to find reads mapping to that gene.

In [104]:
# the location of DEFB125 gene in human genome, gencode.v27 annotation
gene = ('chr20', 87249, 97094)

To start exploring reads from a SAM/BAM file, let's first load the file.

In [105]:
import pysam

# load BAM file
bamfile = pysam.AlignmentFile("aligned/sample_01_accepted_hits.bam", "rb")

Note the 'rb' parameter indicates to read the file as binary, which is the BAM format (BAM stands for Binary Alignment Map). If loading a SAM file, this parameter does not need to be specified

An important and very common application is to count the number of reads (aligned fragments) that overlap a give feature (i.e. region of the genome or gene). One simple approach to doing this is to make a list of all reads generated and simply iterate over the reads to identify whether a read overlaps a region. In this case we need an overlap method that can compare a simple interval (defined by a tuple of chr, start, end) with a pysam AlignedSegment object. This overlap method needs the AlignmentFile object to decode chromosome name, otherwise it is very similar to the above overlap() function.

In [106]:
def overlap(x, gene, bamfile):
    """
    A modified version of overlap that takes an interval and a pysam
    AlignedSegment and tests for overlap
    """
    new_start = max(x.reference_start, gene[1])
    new_end = min(x.reference_end, gene[2])
    
    if (new_start < new_end and bamfile.getrname(x.tid) == gene[0]):
        return True
    
    return False

In [107]:
%%time

# code to iterate over reads and count for a single gene
naive_gene_count = 0

global ref

for x in bamfile.fetch():
    # Note x is of type pysam.AlignedSegment
    if(overlap(x, gene, bamfile)):
        naive_gene_count += 1
    ref = x.reference_end
    

CPU times: user 2.27 s, sys: 15.5 ms, total: 2.29 s
Wall time: 2.29 s


In [108]:
print(naive_gene_count)

438


The problem with this approach is that to do this, you will need to store the entire file in memory. Worse than that, in order to find the reads within a single gene, you would need to iterate over the entire file, which can contain hundreds of millions of reads. This is quite slow even for a single gene, but will only increase if you want to look at many genes.

Instead, more efficient datastructures (and indexing schemes) can be used to retrieve reads based on positions in more efficient ways.

Now that you have access to all of the methods in the Pysam package. Let's take a tour of some of the things you can do in Pysam. We'll start by counting the number of reads over the same region above, but in a smarter way.

(Note: the exact way Pysam, Picard and other tools implement this is based on a similar concept to the interval tree, but generally involve a tree built on a file index so as to avoid reading the entire file into memory, but for the purposes of this discussion the implementation details are not important).

To get the reads overlapping a given region, Pysam has a method called fetch(), which takes the genomic coordinates and returns an iterator of all reads overlapping that region.

In [109]:
bam_iter = bamfile.fetch(gene[0], gene[1], gene[2])

In [110]:
%%time

pysam_gene_count = 0

for x in bam_iter:
    pysam_gene_count += 1

CPU times: user 1.52 ms, sys: 1.21 ms, total: 2.74 ms
Wall time: 1.54 ms


In [111]:
print(pysam_gene_count)

438


Not surprisingly, they are the same - the big difference is that the second method is significantly faster. The reason for this is that Pysam (and Samtools, Picard, and other similar packages) make use of clever tricks to index the file by genomic positions to more efficiently search for reads within a given genomic interval.

Now that we know how to count reads overlapping a region, we can write this as a function and try and compute this for all genes.

In [112]:
def read_count(gene, bamfile):
    """
    Compute the number of reads contained in a bamfile that overlap
    a given interval
    """
    bam_iter = bamfile.fetch(gene[0], gene[1], gene[2])

    pysam_gene_count = 0
    
    for x in bam_iter:
        pysam_gene_count += 1
        
    return pysam_gene_count

You can run this with any gene tuple now

In [113]:
# the location of ZNFX1 gene in human genome, gencode.v27 annotation
gene2 = ("chr20", 49237945, 49278426)

read_count(gene2, bamfile)

2282

Now, let's compute the gene counts for all genes. We'll start by reading in all genes

In [118]:
%%time

gene_counts={}

with open('gencode.v27.chr20.bed', 'r') as f:
    for line in f:
        tokens = line.split('\t')
        gene_local = (tokens[0], int(tokens[1]), int(tokens[2]))
        count = read_count(gene_local, bamfile)
        gene_counts.update({tokens[3].rstrip() : count})   

CPU times: user 3.37 s, sys: 47.1 ms, total: 3.42 s
Wall time: 3.42 s


Now it's easy to query the gene counts for different genes.

In [121]:
print(gene_counts.get("ARFGEF2"))
print(gene_counts.get("SOX12"))
print(gene_counts.get("WFDC8"))

3369
1665
957


## Within sample normalization

Raw read counts are dependent on gene expression as well as gene length. If we want to compare gene level abundances within a single sample, we need to normalize counts by the length of the gene.

The normalization method that we will implement is called **TPM** (transcripts per million) and it allows for comparison of features with different length **within** a sample but not **between** samples. Between sample normalization will be the subject of the following lecture.

In [122]:
import pandas as pd

def tpm(genefile, bamfile):
    """
    Compute the TPM (transcripts per million) metric for all genes within the genefile using
    an RNA-Seq bamfile
    """
    total_mapped_reads = 0
    
    # here you want all reads
    bam_iter = bamfile.fetch()
    
    for x in bam_iter:
        total_mapped_reads += 1

    gene_counts = {}
    with open(genefile, 'r') as f:
        for line in f:
            tokens = line.split('\t')
            gene_local = (tokens[0], int(tokens[1]), int(tokens[2]))
            gene_length = gene_local[2] - gene_local[1]
            raw_count = read_count(gene_local, bamfile)
            if tokens[3] in gene_counts:
                gene_counts[tokens[3]] = (gene_counts[tokens[3]][0] + raw_count, gene_length)
            else:
                gene_counts.update({tokens[3].rstrip() : (raw_count, gene_length)})   
    
    countsDF = pd.DataFrame(gene_counts, index=['raw_count', 'gene_length'])
    countsDF = countsDF.T
    X=countsDF.iloc[:,0].values
    l=countsDF.iloc[:,1].values
    countsDF = countsDF.assign(tpm = 1e6 * (X/l) / (X/l).sum())
    return(countsDF)

Now, lets give it a try

In [123]:
%%time

gene_counts = tpm('gencode.v27.chr20.bed', bamfile)

CPU times: user 4.71 s, sys: 66.3 ms, total: 4.78 s
Wall time: 4.79 s


In [139]:
gene_counts.iloc[sample(range(gene_counts.shape[0]), 20)]

,raw_count,gene_length,tpm
RNU4ATAC7P,54,125,1095.548656
Z98752.1,243,1561,394.776502
OGFR-AS1,825,4961,421.728203
AL031651.1,2216,1438,3908.038141
RNA5SP482,34,118,730.709634
U3,0,203,0.000000
RPLP0P1,117,361,821.914389
LINC00029,456,2812,411.241988
CHGB,1156,13932,210.422556
TRERNA1,146,789,469.271064


In [151]:
def rpkm(genefile, bamfile):
    """
    Compute the RPKM (reads per kilobase per million mapped reads) metric for all genes within the genefile using
    an RNA-Seq bamfile
    """
    total_mapped_reads = 0
    
    # here you want all reads
    bam_iter = bamfile.fetch()
    
    for x in bam_iter:
        total_mapped_reads += 1

    gene_counts = {}
    with open(genefile, 'r') as f:
        for line in f:
            tokens = line.split('\t')
            gene_local = (tokens[0], int(tokens[1]), int(tokens[2]))
            gene_length=gene_local[2]-gene_local[1]
            rpk_metric = 1e9 * read_count(gene_local, bamfile) \
                            / (total_mapped_reads * gene_length)
            gene_counts.update({tokens[3].rstrip() : rpk_metric})   
    return(gene_counts)

In [152]:
%%time

gene_counts_rpkm = rpkm('gencode.v27.chr20.bed', bamfile)

CPU times: user 4.65 s, sys: 55 ms, total: 4.71 s
Wall time: 4.71 s


In [158]:
print(gene_counts_rpkm.get('OGFR-AS1'))

162.84688065616749
